In [1]:
import cv2

In [2]:
# 이미지 불러오기
image = cv2.imread("../dataset_dir/13.한국어글자체/01.손글씨/01_handwriting_sentence_images/1_sentence/00000002.png")
image2 = image.copy() # 박스 그리기
# image3 = image2.copy() # 이미지 자르기

# 컨투어 찾기 전 이미지 전처리
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
gray = cv2.bitwise_not(gray) # 객체보다 배경이 밝은 경우 이미지 반전

# threshold 등 다양한 전처리 시도 -> 객체와 배경을 가장 잘 분리하는 전처리 사용
_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

thresh = cv2.erode(thresh, None, iterations=2)
thresh = cv2.dilate(thresh, None, iterations=2)

# 윤곽선 찾기 및 박스 추출
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

boxes = []
for contour in contours:
  x, y, w, h = cv2.boundingRect(contour) 
  boxes.append({
    "x": x,
    "y": y,
    "w": w,
    "h": h,
  })

boxes = sorted(boxes, key=lambda boxes: boxes["x"])

# 결과 이미지 출력
for box in boxes:
  cv2.rectangle(
    image, 
    (box["x"], box["y"]), 
    (box["x"] + box["w"], box["y"] + box["h"]), 
    (0, 255, 0), 
    2)

In [3]:
boxes

[{'x': 55, 'y': 50, 'w': 39, 'h': 55},
 {'x': 117, 'y': 45, 'w': 17, 'h': 86},
 {'x': 229, 'y': 35, 'w': 51, 'h': 52},
 {'x': 247, 'y': 90, 'w': 53, 'h': 50},
 {'x': 345, 'y': 52, 'w': 32, 'h': 47},
 {'x': 392, 'y': 61, 'w': 25, 'h': 51},
 {'x': 456, 'y': 42, 'w': 36, 'h': 57},
 {'x': 488, 'y': 90, 'w': 60, 'h': 35},
 {'x': 519, 'y': 39, 'w': 22, 'h': 46},
 {'x': 565, 'y': 54, 'w': 49, 'h': 59},
 {'x': 592, 'y': 54, 'w': 7, 'h': 5},
 {'x': 618, 'y': 53, 'w': 26, 'h': 63},
 {'x': 671, 'y': 37, 'w': 8, 'h': 17},
 {'x': 691, 'y': 42, 'w': 7, 'h': 10},
 {'x': 733, 'y': 46, 'w': 44, 'h': 72},
 {'x': 789, 'y': 50, 'w': 35, 'h': 76},
 {'x': 908, 'y': 34, 'w': 8, 'h': 16},
 {'x': 925, 'y': 33, 'w': 9, 'h': 16},
 {'x': 947, 'y': 104, 'w': 58, 'h': 26},
 {'x': 958, 'y': 46, 'w': 48, 'h': 43},
 {'x': 975, 'y': 131, 'w': 36, 'h': 16},
 {'x': 1106, 'y': 35, 'w': 31, 'h': 13},
 {'x': 1107, 'y': 58, 'w': 40, 'h': 37},
 {'x': 1125, 'y': 97, 'w': 37, 'h': 50},
 {'x': 1201, 'y': 52, 'w': 55, 'h': 90},
 

In [4]:
# 결과 이미지 출력
cv2.imwrite(f'./image/box_group.jpg', image)

True

In [5]:
import numpy as np

min_distance_value = 35.99999999999

Primary_group_box_list = []

for rangeNum in range(1, len(boxes)):
    box1 = boxes[rangeNum-1]
    box2 = boxes[rangeNum]

    coordinate_box1 = (
        box1["x"], # X 좌표
        box1["y"], # Y 좌표
        box1["x"] + box1["w"], # W 좌표
        box1["y"] + box1["h"] # H 좌표
    )
    
    coordinate_box2 = (
        box2["x"], # X 좌표
        box2["y"], # Y 좌표
        box2["x"] + box2["w"], # W 좌표
        box2["y"] + box2["h"] # H 좌표
    )


    box1_center_x = (box1["x"] + box1["w"]) / 2
    box1_center_y = (box1["y"] + box1["h"]) / 2


    box2_center_x = (box2["x"] + box2["w"]) / 2
    box2_center_y = (box2["y"] + box2["h"]) / 2

    # 두 박스의 중심점 간의 거리를 계산합니다.
    distance = np.sqrt(
        ((box1_center_x - box2_center_x)**2) +
        ((box1_center_y - box2_center_y)**2)
    )

    print(box1, box2, distance, distance < min_distance_value)

    if distance < min_distance_value:
        setGrouping = {
            "x": min(coordinate_box1[0], coordinate_box2[0]), # X 좌표
            "y": min(coordinate_box1[1], coordinate_box2[1]), # Y 좌표
            "w": max(coordinate_box1[2], coordinate_box2[2]), # W 좌표
            "h": max(coordinate_box1[3], coordinate_box2[3]) # H 좌표
        }
        
        Primary_group_box_list.append(setGrouping)

{'x': 55, 'y': 50, 'w': 39, 'h': 55} {'x': 117, 'y': 45, 'w': 17, 'h': 86} 23.853720883753127 True
{'x': 117, 'y': 45, 'w': 17, 'h': 86} {'x': 229, 'y': 35, 'w': 51, 'h': 52} 76.24303246854758 False
{'x': 229, 'y': 35, 'w': 51, 'h': 52} {'x': 247, 'y': 90, 'w': 53, 'h': 50} 28.324018076537094 True
{'x': 247, 'y': 90, 'w': 53, 'h': 50} {'x': 345, 'y': 52, 'w': 32, 'h': 47} 43.617656975128774 False
{'x': 345, 'y': 52, 'w': 32, 'h': 47} {'x': 392, 'y': 61, 'w': 25, 'h': 51} 21.02974084481309 True
{'x': 392, 'y': 61, 'w': 25, 'h': 51} {'x': 456, 'y': 42, 'w': 36, 'h': 57} 38.05916446797013 False
{'x': 456, 'y': 42, 'w': 36, 'h': 57} {'x': 488, 'y': 90, 'w': 60, 'h': 35} 30.870698080866262 True
{'x': 488, 'y': 90, 'w': 60, 'h': 35} {'x': 519, 'y': 39, 'w': 22, 'h': 46} 20.303940504246953 True
{'x': 519, 'y': 39, 'w': 22, 'h': 46} {'x': 565, 'y': 54, 'w': 49, 'h': 59} 39.092838218783754 False
{'x': 565, 'y': 54, 'w': 49, 'h': 59} {'x': 592, 'y': 54, 'w': 7, 'h': 5} 28.02231253840411 True
{'x

In [6]:
Primary_group_box_list

[{'x': 55, 'y': 45, 'w': 134, 'h': 131},
 {'x': 229, 'y': 35, 'w': 300, 'h': 140},
 {'x': 345, 'y': 52, 'w': 417, 'h': 112},
 {'x': 456, 'y': 42, 'w': 548, 'h': 125},
 {'x': 488, 'y': 39, 'w': 548, 'h': 125},
 {'x': 565, 'y': 54, 'w': 614, 'h': 113},
 {'x': 618, 'y': 37, 'w': 679, 'h': 116},
 {'x': 671, 'y': 37, 'w': 698, 'h': 54},
 {'x': 733, 'y': 46, 'w': 824, 'h': 126},
 {'x': 908, 'y': 33, 'w': 934, 'h': 50},
 {'x': 947, 'y': 46, 'w': 1006, 'h': 130},
 {'x': 958, 'y': 46, 'w': 1011, 'h': 147},
 {'x': 1106, 'y': 35, 'w': 1147, 'h': 95},
 {'x': 1107, 'y': 58, 'w': 1162, 'h': 147},
 {'x': 1319, 'y': 51, 'w': 1380, 'h': 102},
 {'x': 1324, 'y': 51, 'w': 1393, 'h': 145},
 {'x': 1505, 'y': 44, 'w': 1590, 'h': 129},
 {'x': 1549, 'y': 43, 'w': 1590, 'h': 129},
 {'x': 1565, 'y': 43, 'w': 1658, 'h': 112},
 {'x': 1620, 'y': 53, 'w': 1698, 'h': 134},
 {'x': 1792, 'y': 42, 'w': 1894, 'h': 136},
 {'x': 1835, 'y': 41, 'w': 1894, 'h': 136},
 {'x': 1911, 'y': 40, 'w': 2020, 'h': 126},
 {'x': 1970, '

In [7]:
# 결과 이미지 출력
for box in Primary_group_box_list:
# box = group_box_list[0]
  cv2.rectangle(
    image2, 
    (box["x"], box["y"]), 
    (box["w"], box["h"]), 
    (0, 255, 0), 
    2)

In [8]:
Primary_group_box_list[0]

{'x': 55, 'y': 45, 'w': 134, 'h': 131}

In [9]:
# 결과 이미지 출력
cv2.imwrite(f'./image/box_group_end.jpg', image2)

True

In [11]:
import cv2

box = Primary_group_box_list[0]
output = image2[box["x"]:box["y"], box["w"]:box["h"]]

# 결과 이미지 출력
cv2.imwrite(f'./image/box.jpg', output)

error: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:786: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'
